In [23]:
import derive
reload(derive)

fun = lambda x: (3 * x[0] + x[1]) ** 2

print derive.hess(fun)([3, 1])

[[ 18.00160021   6.0005334 ]
 [  6.0005334    2.00373051]]


In [1]:
import sympy as sm

D = 2
x = sm.symbols(' '.join(['x%s' % i for i in xrange(D)]))
fun = (3 * x[0] + x[1]) ** 2

point = [3, 12]
partials = []
for i in xrange(D):
    rest = range(D)
    del rest[i]
    vars = [x[i] for i in rest]
    vals = [point[i] for i in rest]
    substitutions = {var: val for (var, val) in zip(vars, vals)}
    partials.append(sm.diff(fun.subs(substitutions), x[i]).subs(x[i], point[i]))
    
print partials

[0, 0]


In [25]:
import derive
reload(derive)

import tensorflow as tf      

x = tf.Variable(0, dtype = tf.float32)
y = tf.Variable(0, dtype = tf.float32)
f = (3 * x + y) ** 2
hess = derive.hess(f)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print hess([x, y]).eval()

[[ 18.   6.]
 [  6.   2.]]


In [211]:
import theano
import theano.tensor as T

x = T.dvector('x')
y = x ** 2
cost = y.sum()
gy = T.grad(cost, x)
H, updates = theano.scan(lambda i, gy,x : T.grad(gy[i], x), sequences=T.arange(gy.shape[0]), non_sequences=[gy, x])
f = theano.function([x], H, updates=updates)